# Entity Resolution
## Familiarization with Dedupe Library

This notebook is basically notes taken from reading through the dedupe documentation as well as other resources to solidfy the understanding of entity resolution with active machine learning.

Reference: Gregg, Forest and Derek Eder. 2015. Dedupe. https://github.com/dedupeio/dedupe

## Overview of Dedupe Library 
The dedupe library can:
1. Remove duplicate entries from a spreadsheet of names and addresses
2. Link a list of (customer) information to another (order) history, even without unique (customer) id’s
3. Take a database of campaign contributions and figure out which ones were made by the same person, even if the names were entered slightly differently for each record
4. Takes in human training data and comes up with the best rules for your dataset to quickly and automatically find similar records, even with very large databases.

Features:
1. Machine learning – reads in human labeled data to automatically create optimum weights and blocking rules
2. Runs on a laptop – makes intelligent comparisons so you don’t need a powerful server to run it
3. Built as a library – so it can be integrated into applications or import scripts
4. Extensible – supports adding custom data types, string comparators and blocking rules
5. Open source

first name | last name | address                  | phone   
:----------|:----------|:-------------------------|:---------
bob        | roberts   | 1600 pennsylvania ave.   | 555-0123 
Robert     | Roberts   | 1600 Pensylvannia Avenue |          


If we have to decide which records in our data are about the same person or organization, then we could just go through by hand, compare every record, and decide which records are about the same entity.

Dedupe is a software library that can make these decisions about whether records are about the same thing about as good as a person can, but quickly.

## Record similarity

One way that people have approached this problem is by saying that records that are more similar are more likely to be duplicates. One way to measure this is called string metric. A string metric is a way of taking two strings and returning a number that is low if the strings are similar and high if they are dissimilar. One famous string metric is called the Hamming distance. It counts the number of substitutions that must be made to turn one string into another.

Dedupe uses a metric called **Affine Gap Distance**.

#### Off Tangent: Gap Distance
- Definition of **gap**: Any maximal, consecutive run of spaces in a single sequence of a given alignment
- Definition of **length**: The number of indel operations in it.

Considering the following alignment:
```
S = a t t c - - g a - t g g a c c
T = a - - c g t g a t t - - - c c
```
- This alignment has **four gaps** containing a total of eight spaces.
- This alignment would be described as having **seven matches**, **no mismatch**, **four gaps** and **eight spaces**.

##### **Constant**

- This is the simplest type of gap penalty: a fixed negative score is given to every gap, <u>regardless of its length.</u>
- This <u>encourages the algorithm to make fewer, larger, gaps</u> leaving larger contiguous sections.

```
ATTGACCTGA
||   |||||
AT---CCTGA
```
> Aligning two short DNA sequences, with `-` depicting a gap of one base pair. If each match was worth 1 point and the whole gap -1, the total score: 7 − 1 = 6. <br>

##### **Linear**

- Compared to the constant gap penalty, the linear gap penalty <u>takes into account the length (L) of each insertion/deletion in the gap</u>.
- Therefore, if the penalty for each inserted/deleted element is `B` and the length of the gap `L`; the total gap penalty would be the product of the two `BL`. 
- This method <u>favors shorter gaps, with total score decreasing with each additional gap.</u>

```
ATTGACCTGA
||   |||||
AT---CCTGA
```
> Unlike constant gap penalty, the size of the gap is considered. With a match with score 1 and each gap -1, the score here is (7 − 3 = 4).

##### <u>Affine Gap Penalty Model</u>

- Most widely used gap penalty function
- The affine gap penalty combines the components in both the **constant** and **linear gap penalty**, taking the form $ A + B.L $
    - $A$: gap opening penalty - refers to the cost required to "open a gap"
    - $B$: gap extension penalty - the cost to "extend the length" of an existing gap by 1
    - $L$: length of the gap
- Note that the Affine Gap Penalty Model is simply the Constant Gap Weight Model when $B = 0$.
- In general:
    - if the interest is to find closely related matches (e.g. removal of vector sequence during genome sequencing), a higher gap penalty should be used to reduce gap openings
    - On the other hand, gap penalty should be lowered when interested in finding a more distant match.
- The relationship between A and B also have an effect on gap size.
    - If the size of the gap is important, a small A and large B (more costly to extend a gap) is used and vice versa
    - Only the ratio A/B is important, as multiplying both by the same positive constant k will increase all penalties by k: kA+kBL = k(A+BL) which does not change the relative penalty between different alignments.
    
References:
- https://en.wikipedia.org/wiki/Gap_penalty 
- https://web.archive.org/web/20130626060959/http://www.biogem.org/downloads/notes/Gap%20Penalty.pdf

#### Record by Record or Field by Field?

1. We can either calculate similarities of two records as two long strings or as field by field.
2. The advantage to Field by Field is that we can add varying numeric weightages to each field

```
record_distance = (0.5 * string_distance('bob', 'Robert')
                   + 2.0 * string_distance('roberts', 'Roberts')
                   + 2.0 * string_distance('1600 pennsylvania ave.', '1600 Pensylvannia Avenue')
                   + 0.5 * string_distance('555-0123', ''))
```

## Regularized Logistic Regression

1. Record distance / metric (eg we obtain a distance score of `8`), the number does not really translate into practical insights as to whether the two records are duplicates or not. 
    > `8` could mean the two records are duplicates, or not. We don't know
2. If we supply pairs of records that we label as either being duplicates or distinct to a logistic regression model, the model can then learn a set of weights such that the record distance can be easily transformed into our best estimate of the probability that a pair of records are duplicates.

### Active learning

1. Dedupe saves the users' time by making good use of the labeling time through Active Learning.
2. It keeps track of a bunch of unlabeled pairs and whether:
    - the current learning blocking rules would cover the pairs
    - the current learned classifier would predict that the pairs were duplicates or distinct
3. It also maintains a set of pairs where there is disagreement:
    - Pairs which classifier believes were duplicates but were not covered by the current blocking rules
    - Pairs which the classifier believes are distinct but which are blocked together.
        > - Dedupe then randomly picks a pair of records from this disagreement set and asks the user to decide.
        > - Once the user provides the label, dedupe relearns the weights and blocking rules
        > - It then recalculates the disagreement set.

### Smart Comparisons

- The naive approach of comparing two records at a time is time consuming. 
    - Say we have 30,000 records, we have to process $\frac{30,000 * 29,999}{2} = 449,985,000 $ (~450 mil) entries!! 
    - Assuming that our machine is able to make $\frac{10,000 \ comparisons}{second}$, it will take 12 hours to go through each of them!

#### Blocking
- Duplicate records almost always share *something* in common. 
- If we define groups of data that share something and only compare the records in that group / block, then we will dramatically reduce the number of comparisons we will make.
- Dedupe approaches it in two ways: predicate blocks and canopies.

##### <u>**Predicate blocks**</u>
- A predicate block is a bundle of records that all share a feature – a feature produced by a simple function called a predicate.
- Predicate functions take in a record field, and output a set of features for that field.
    - These features could be “the first 3 characters of the field,” “every word in the field,” and so on
    - Records that share the same feature become part of a block.
- Let's take the below as an example, using a "first 3 character" predicate on the `address` field below:

first name | last name | address | phone | record_id
:----------|:----------|:--------|:------|:-----------
bob | roberts | 1600 pennsylvania ave. | 555-0123 | 1
Robert |Roberts |1600 Pensylvannia Avenue | |2
steve | Jones | 123 Cowabunga Lane | 555-0000 | 3 
Stephen | Janes | 123 Cawabunga Ln | 444-555-0000 | 4



- From four records, we now reduced it down to two blocks:

```python
{ '160' : (1,2) # tuple of record_id'
  '123' : (3,4)
  }
```

- Here, we’re applying the “first 3 characters” predicate function to the `address` field in our data, the function outputs the following features – 
    - `160, 160, 123, 123`.
    - we group together the records that have identical features into “blocks”
    
- Other simple predicates that Dedupe uses:
    - whole field
    - token field
    - common integer
    - same three char start
    - same five char start
    - same seven char start
    - near integers
    - common four gram
    - common six gram

#### Index Predicates / Blocks
- Dedupe also uses another way of producing blocks from searching and index.
- It will use a special data structure, like an inverted index that allows us to quickly find records similar to target records.
- Dedupe uses another way of producing blocks from searching and index.
    - Populate the index with all the unique values that appear in that field
- When blocking, for each record, we search the index for values similar to the record's field. We then block together the records that share at least one common search result.
- Index predicates require building an index from all unique values in a field.
- Index predicates are usually slower than predicate blocking as building an index from all unique values in a field can be resource consuming.

#### Combining Blocking Rules
- Dedupe can also combine a few blocks. This is called *disjunctive blocks*.
    - eg: combining blocks of records based on "city" field, but also combining it with "zip code"
    - Dedupe tries these cross-field blocks.
    
#### Learning good blocking rules for a given data
- Having a large set of predicates, Dedupe would try to combine them into a smaller set of combined blocking rules that generalizes to the labeled duplicated pairs.
- However, there can be a large possible combinations to choose from.
- The goal is then to find a small set of rules that generalizes well to every labeled duplicated pair, but also minimizes the total number of pairs that Dedupe has to compare.
- To approach this problem, Dedupe uses greedy algorithms, in particular Chvatal's Greedy Set-Cover algorithm.

### Clustering Duplicates (Grouping beyond pairwise probabilities)

- Even though we may have calculated the probabilities for each of the pairs of record of being duplicates (or not), we still need to ensure that it is not just pairs of records that can be duplicates. 
- Three, four, or even thousands of records could all refer to the same entity, but we only have pairwise measures.

- Assuming we have the following pairwise probabilities between recrods A, B, and C.

``` 
A -- 0.6 --- B --- 0.6 -- C 
```

> - The probability that A & B are duplicates has been calculated 60% and the probability that B & C are duplicates has also been calculated as 60%.
> - What is the probability that A & C are duplicates?

- Dedupe uses **hierarchical clustering with centroid linkage** to give us clusters of linkage
    - What it does is that all the points within some distance of a particular centroid are part of the same group.
    - In the example above, B is the centroid and therefore A, B and C would be put in the same group.
- Downside to clustering alogrithms is that they depend on us setting the threshold for group membership 
    - Threshold of the distance of a point to a particular centroid where the point is considered to be part of the centroid's cluster
- This is not the most ideal as pairwise probabilities are not transitive.
- In recent years, there are other researches that solves the problem of turning pairwise distances into clusters by avoiding making pairwise comparisons altogether. But these are not compatible with Dedupe's main principal pairwise approach.
    -  <a href=http://people.cs.umass.edu/~sameer/files/hierar-coref-acl12.pdf> Michael Wick, et.al, 2012. “A Discriminative Hierarchical Model for Fast Coreference at Large Scale”</a>
    -  <a href=http://arxiv.org/abs/1312.4645> Rebecca C. Steorts, et. al., 2013. “A Bayesian Approach to Graphical Record Linkage and De-duplication” </a>

### Precision and Recall
<img src="references/525px-Precisionrecall.svg.png">

- Precision is the fraction of the relevant instances among the retrieved instances.
- Recall is the fraction of the relevant instances that were retrieved.

##### Example: Cats vs Dogs 
- Consider image recognition model in recognizing dogs (the **relevant** element) from a set of images containing 10 cats and 12 dogs
    - Total "positives" retrieved (classified by model as dogs): 8
        - 8 were truly dogs (True Positives)
        - 3 were actually non-dogs  (False Positives)
    - Total "negatives" retrieved (classified by model as cats / non-dogs): 14
        - 7 were truly non-dogs / cats (True Negatives)
        - 7 were actually dogs (False Negatives)

$$ 
Precision = \frac{True\ Positives}{True\ Positives + False\ Positives}
$$

$$ 
Recall = \frac{True\ Positivies}{True\ Positives + False\ Negatives}
$$


### F-Score

- The traditional F-measure / balanced F-score (**$F_1 score$**) is the harmonic mean of precision and recall:
$$
{\displaystyle F_{1}={\frac {2}{\mathrm {recall^{-1}} +\mathrm {precision^{-1}} }}=2\cdot {\frac {\mathrm {precision} \cdot \mathrm {recall} }{\mathrm {precision} +\mathrm {recall} }}={\frac {\mathrm {tp} }{\mathrm {tp} +{\frac {1}{2}}(\mathrm {fp} +\mathrm {fn} )}}}.
$$

- $F_\beta$ is a general F score, where $\beta$ is a positive real factor, chosen such that recall is considred $\beta$ times as important as precision:
$$
F_\beta = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}}
$$
    > In terms of Type I and Type II errors:
    $$
    F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}\,
    $$


- Two commonly used values for $\beta$ are 2, which weights recall higher than precision, and 0.5, which weights recall lower than precision.

### Choosing a good threshold 

- Using Precision and Recall (and F-score), we can determine if the pairwise probabilities that were calculated by Dedupe are true "duplicates" or false "duplicates". 
- It is, therefore, very important that we find a good threshold (F-score) that is optimal for our priorities (determining the trade-offs between precision and recall).
    - Typically, for us to find a good threshold, we need to look at the true precision and recall values of some data where we have already known their true values (pairs that are truly duplicates and pairs that are truly non-duplicates)
    - However, we will only get a good treshold if the labeled exampls are representative of the data that we are trying to classify.
- In active learning step, the labeled examples that we make with Dedupe are not representative by design.
    - We are not trying to find the most representative data examples. 
    - We are trying to find the ones that will teach us the most.
- The approach that Dedupe takes is:
    1) Take a random sample of blocked data
    1) Calculate the pairwise probability that records would be duplicates within each block
    1) From these probabilities, we can calculate the expected number of duplicates and distinct pairs
    1) Calculate the expected precision and recall

### Special Cases

- The process we have been describing is for the most general case–when you have a dataset where an arbitrary number of records can all refer to the same entity.
- There are certain special cases where we can make more assumptions about how records can be linked, which if true, make the problem much simpler.
- One important case we call Record Linkage. 
    - Say you have two datasets and you want to find the records in each dataset that refer to the same thing. 
    - If you can assume that each dataset, individually, is unique, then this puts a big constraint on how records can match. 
    - If this uniqueness assumption holds, 
        - then (A) two records can only refer to the same entity if they are from different datasets and 
        - (B) no other record can match either of those two record

#### Scratch Notes

- Other field distances: https://docs.dedupe.io/en/latest/Variable-definition.html
- Hierarchical clustering with centroid linkage 
- Researches about clustering pairwise distances without making pairwise comparisons altogether:
    -  <a href=http://people.cs.umass.edu/~sameer/files/hierar-coref-acl12.pdf> Michael Wick, et.al, 2012. “A Discriminative Hierarchical Model for Fast Coreference at Large Scale”</a>
    -  <a href=http://arxiv.org/abs/1312.4645> Rebecca C. Steorts, et. al., 2013. “A Bayesian Approach to Graphical Record Linkage and De-duplication” </a>

# Testing out Dedupe - Example 1: `csv_example`

In [25]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

from pathlib import Path

In [21]:
def preProcess(column):
    '''
    Do a little bit of data cleaning with the help of Unidecode and Regex. 
    Things like casing, extra spaces, quotes and new lines can be ignored
    '''
    
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    
    if not column:
        column = None
    return column

In [22]:
def readData(filename):
    '''
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    '''
    
    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)
    
    return data_d            

In [23]:
# set logging 
log_level = logging.DEBUG ## Verbosity multiple times
logging.getLogger().setLevel(log_level)

In [26]:
data_path = Path(r"./dedupe-examples/csv_example/")

input_file = data_path / r"csv_example_messy_input.csv"
output_file = data_path / r"csv_example_output.csv"
setting_file = data_path / r"csv_example_learned_settings"
training_file = data_path / r"csv_example_training.json"

In [30]:
data_d = readData(input_file)

In [31]:
# define fields for dedupe

fields = [
    {"field": "Site name", "type": "String"},
    {"field": "Address", "type": "String"},
    {"field": "Zip", "type": "Exact", "has missing": True},
    {"field": "Phone", "type": "String", "has missing": True},
]

In [32]:
#initialize Dedupe object 

deduper = dedupe.Dedupe(fields)

In [33]:
# Prepare training
deduper.prepare_training(data_d)

DEBUG:root:1350 blocked samples were requested, but only able to sample 1348
DEBUG:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.6, Address)
DEBUG:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.2, Address)
DEBUG:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.4, Address)
DEBUG:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.8, Address)
DEBUG:dedupe.blocking:Canopy: LevenshteinCanopyPredicate: (4, Address)
DEBUG:dedupe.blocking:Canopy: LevenshteinCanopyPredicate: (3, Address)
DEBUG:dedupe.blocking:Canopy: LevenshteinCanopyPredicate: (1, Address)
DEBUG:dedupe.blocking:Canopy: LevenshteinCanopyPredicate: (2, Address)
INFO:dedupe.canopy_index:Removing stop word  s
DEBUG:dedupe.blocking:Canopy: TfidfNGramCanopyPredicate: (0.4, Address)
DEBUG:dedupe.blocking:Canopy: TfidfNGramCanopyPredicate: (0.8, Address)
DEBUG:dedupe.blocking:Canopy: TfidfNGramCanopyPredicate: (0.2, Address)
DEBUG:dedupe.blocking:Canopy: TfidfNGramCanopyPredicate: (0.6, Address)
DEBUG:dedupe.blockin

In [34]:
dedupe.console_label(deduper)

DEBUG:dedupe.labeler:Classifier: 0.97, Covered: False
Site name : centers for new horizons altgeld
Address : 941 e. 132nd st
Zip : 60627
Phone : 4683055

Site name : centers for new horizons altgeld ii
Address : 941 e. 132nd st
Zip : 60627
Phone : 4683055

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


 y


DEBUG:dedupe.labeler:Classifier: 0.97, Covered: False
Site name : children's dev inst morgan pk
Address : 10928 s. halsted
Zip : 60628
Phone : 9950600

Site name : children's dev inst morgan pk i/t
Address : 10928 s. halsted
Zip : 60628
Phone : 9950600

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, Address)
DEBUG:dedupe.labeler:Classifier: 0.97, Covered: False
Site name : henry booth house - little hands & feet
Address : 7801 s wolcott ave
Zip : None
Phone : None

Site name : henry booth house little hands & feet
Address : 7801 s wolcott
Zip : 60620
Phone : 9948561

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

DEBUG:dedupe.labeler:Classifier: 0.52, Covered: False
Site name : st. augustine college - st. augustine - nayarit
Address : 2610 w 25th pl
Zip : None
Phone : 8783641

Site name : saint augustine college-nayarit head start
Address : 2610 west 25th place lower level
Zip : 60608
Phone : 8783653

19/10 positive, 6/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 u


DEBUG:dedupe.labeler:Classifier: 0.58, Covered: False
Site name : south east asia center - south east asia center(foster)
Address : 1112 w foster ave
Zip : None
Phone : None

Site name : catholic charities chicago - ebenezer
Address : 1652 w foster avenue
Zip : 60640
Phone : 8789925

19/10 positive, 6/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling


In [35]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.010000, score 0.46895255686615833
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, Phone), TfidfTextCanopyPredicate: (0.4, Address), SimplePredicate: (commonSixGram, Site name))
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, Address), SimplePredicate: (suffixArray, Site name), SimplePredicate: (commonThreeTokens, Site name))


In [37]:
with open(training_file, "w") as tf:
    deduper.write_training(tf)

with open(setting_file, "w") as sf:
    deduper.write_training(sf)

In [38]:
#### Clustering
# partition will return sets of records that dedupe believes are all referring to the same entity
clustered_dupes = deduper.partition(data_d, 0.5)

print("# duplicate sets:", len(clustered_dupes))

DEBUG:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.4, Address)
DEBUG:dedupe.api:matching done, begin clustering


# duplicate sets: 1621


In [39]:
cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

In [44]:
with open(output_file, "w") as f_output, open(input_file) as f_input:
    reader = csv.DictReader(f_input)
    fieldnames = ["Cluster ID", "confidence_score"] + reader.fieldnames
    
    writer = csv.DictWriter(f_output, fieldnames = fieldnames)
    writer.writeheader()
    
    for row in reader:
        row_id = int(row["Id"])
        row.update(cluster_membership[row_id])
        writer.writerow(row)